<a href="https://colab.research.google.com/github/meetarora10/veersa-hack/blob/main/model_recognize_doctor_and_patient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import requests

# url = "https://api.deepgram.com/v1/listen"
# headers = {
#     "Authorization": "0b6d77fe6ccee8475bccfd2b8912749ad2b3846f",
# }
# audio = open("meeting_audio.wav", "rb")

# response = requests.post(url, headers=headers, files={"file": audio}, data={"diarize": "true"})
# transcript = response.json()["results"]["channels"][0]["alternatives"][0]["transcript"]
# utterances = response.json()["results"]["utterances"]


In [ ]:
# def format_dialogue(utterances):
#     dialogue = []
#     for u in utterances:
#         speaker = "Doctor" if u["speaker"] == 0 else "Patient"
#         dialogue.append(f"{speaker}: {u['transcript']}")
#     return "\n".join(dialogue)


In [ ]:
# print(format_dialogue(utterances))

In [ ]:
!pip install pydub speechrecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 12.7 MB/s eta 0:00:00


In [ ]:
import speech_recognition as sr
from pydub import AudioSegment
from pydub.silence import split_on_silence

# Load the audio file
sound = AudioSegment.from_wav("doctor_patient.wav")

# Split on silence
chunks = split_on_silence(sound, min_silence_len=500, silence_thresh=sound.dBFS-14, keep_silence=250)

recognizer = sr.Recognizer()
transcripts = []

for i, chunk in enumerate(chunks):
    chunk.export(f"chunk{i}.wav", format="wav")
    with sr.AudioFile(f"chunk{i}.wav") as source:
        audio = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio)
            transcripts.append(text)
        except sr.UnknownValueError:
            transcripts.append("[Unrecognized]")


In [ ]:
import joblib

# Load the vectorizer and classifier
vectorizer = joblib.load("tfidf_vectorizer.pkl")
clf = joblib.load("logistic_regression_model.pkl")


In [ ]:
# Vectorize the transcripts
X = vectorizer.transform(transcripts)

# Predict labels
labels = clf.predict(X)

# Combine labels with transcripts
dialogue = []
for label, text in zip(labels, transcripts):
    dialogue.append(f"{label}: {text}")
    print(dialogue)


["patient: I've been feeling really weak and feverish for the past few days"]
["patient: I've been feeling really weak and feverish for the past few days", 'doctor: it could be a viral infection but I need to check your vitals to be sure']


In [ ]:
print("\n".join(dialogue))


patient: I've been feeling really weak and feverish for the past few days
doctor: it could be a viral infection but I need to check your vitals to be sure
